## overview

In [1]:
# default package
import logging
import sys 
import os
import pathlib
import IPython
import random

In [2]:
# third party package
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import psycopg2
import sqlalchemy
from sqlalchemy.sql.expression import column,select,table
from sqlalchemy import create_engine
import sklearn.datasets

In [3]:
# my package
sys.path.append(os.path.join(pathlib.Path().resolve(),"../"))

In [4]:
# reload settings
%load_ext autoreload
%autoreload 2

In [5]:
# logger
logger=logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

In [6]:
## grap setting
sns.set()

In [7]:
## db setting
engine = create_engine('postgresql://postgres:postgres@db:5432/postgres')
with engine.connect() as conn:
    print(conn.closed)

False


## sql to python

In [8]:
def clean(engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除
    """
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")

In [9]:
def ct(file:str,chap_num:int,num:int,sub_num:int=None,engine:sqlalchemy.engine=engine)->None:
    """
    pubulic schema内のtableをすべて削除してから、指定のtable作成
    """
    if sub_num==None: 
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}"
    else:
        filename=f"../code-ssql2/ddl/ssql2-{file}{chap_num:02}_{num:02}_{sub_num}"
        
    with open(filename,"r") as f:
        stmt=f.read() 
    with engine.connect() as conn:
        conn.execute("drop schema public cascade; create schema public;")
        conn.execute(stmt)

In [10]:
def exe(stmt:str,engine:sqlalchemy.engine=engine):
    """
    sql実行
    """
    with engine.connect() as conn:
        query=conn.execute(stmt)
        try:
            result=query.fetchall()
            return result
        except:
            return None

In [11]:
def exe_df(stmt:str,engine:sqlalchemy.engine=engine)->pd.DataFrame:
    """
    sql(select)を実行してdataframeを返す
    """
    with engine.connect() as conn:
        df=pd.read_sql(stmt,conn)
    return df 

## List

In [12]:
#11-1
ct("List",11,1)
exe("""
create index 費目IDインデックス on 家計簿(費目ID);
create index メモインデックス on 家計簿(メモ);
""")

In [13]:
#11-2
exe("""
select * from 家計簿
where メモ='1月の給料'
""")

[(datetime.date(2018, 2, 10), 1, '1月の給料', 280000, 0)]

In [14]:
#11-7
ct("List",11,7)
exe("""
create view 家計簿4月 as 
select * from 家計簿
where 日付>'2018-04-01' 
and 日付<'2018-04-30'
""")

In [15]:
#11-9
#連番指定
clean()
exe("""
create table 費目(
id integer primary key
)
""")

In [16]:
#11-12
ct("List",11,1)
exe("""
create sequence 費目シーケンス;
select nextval('費目シーケンス');
""")

[(1,)]